In [9]:
from datasets import load_dataset

# dataset1 = load_dataset("jiandong/crimson-attck-vectors")
# dataset = load_dataset("Xcvddax/ATT")
# dataset3 = load_dataset("jiandong/crimson-attck-bge-vectors")

dataset = load_dataset("Xcvddax/Attack-techniques",split='train')
# dataset2 = load_dataset("dear-daddy/attck",split='train')
 

In [14]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(dataset)
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

TypeError: load_local() missing 1 required positional argument: 'folder_path'

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

In [ ]:
from  langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
  output_texts = []
  for i in range(len(docs)):
      text = f"""id: {docs['id'][i]}\n ,attck_id: {docs['attck_id'][i]} , description: {docs['description'][i]} ,
        kill_chain_phases: {docs['kill_chain_phases'][i]} , domains: {docs['domains'][i]} , tactic_type: {docs['tactic_type'][i]}
        """
      output_texts.append(text)
  return output_texts


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)